In [1]:
import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
import csv
import random
import re
import os
import unicodedata
import codecs
import itertools

In [2]:
CUDA = torch.cuda.is_available()
device = torch.device("cuda" if CUDA else "cpu")

In [3]:
lines_filepath = os.path.join("cornell movie-dialogs corpus", "movie_lines.txt")
conv_filepath = os.path.join("cornell movie-dialogs corpus", "movie_conversations.txt")

In [4]:
with open (lines_filepath, 'r', encoding='utf-8') as file:
    lines = file.readlines()
    for line in lines[:8]:
        print(line.strip()) 

L1045	u0	m0	BIANCA	They do not!
L1044	u2	m0	CAMERON	They do to!
L985	u0	m0	BIANCA	I hope so.
L984	u2	m0	CAMERON	She okay?
L925	u0	m0	BIANCA	Let's go.
L924	u2	m0	CAMERON	Wow
L872	u0	m0	BIANCA	Okay -- you're gonna need to learn how to lie.
L871	u2	m0	CAMERON	No


In [5]:
line_fields = ["lineID", "characterID","movieID","character","text"]
lines = {}
with open (lines_filepath, 'r', encoding='iso-8859-1') as f:
    for line in f:
        values = line.split("\t")
        lineObj = {}
        for i, field in enumerate(line_fields):
            lineObj[field] = values[i]
        lines[lineObj['lineID']] = lineObj

In [6]:
conv_fields = ["character1ID", "character2ID","movieID","utteranceIDs"]
conversations = []
with open (conv_filepath, 'r', encoding='iso-8859-1') as f:
    for line in f:
        values = line.split("\t")
        convObj = {}
        for i,field in enumerate(conv_fields):
            convObj[field] = values[i]        
        lineIds = eval(convObj["utteranceIDs"])
        split_lineIds = [re.findall(r'L\d+', string[0]) for string in lineIds]
        temp = []
        for string in lineIds:
            temp = re.findall(r'L\d+', string)
        
        convObj["lines"] = []
        for lineId in temp:
            convObj["lines"].append(lines[lineId])
        conversations. append(convObj)

In [7]:
qa_pairs = []
for conversation in conversations:
    for i in range(len(conversation["lines"]) -1):
        inputLine = conversation["lines"][i]["text"].replace("\r\r\n","").strip()
        targetLine = conversation["lines"][i+1]["text"].replace("\r\r\n","").strip()
        if inputLine and targetLine:
            qa_pairs.append([inputLine,targetLine])

In [8]:
datafile = os.path.join("cornell movie-dialogs corpus", "formated_movie_lines.txt")
delimiter = '\t'
delimiter = str(codecs.decode(delimiter, "unicode_escape"))

print("\nWriting newly formatted file...")
with open(datafile,'w', encoding='utf-8') as outputfile:
    writer = csv.writer(outputfile, delimiter=delimiter)
    for pair in qa_pairs:
        writer.writerow(pair)
print("Done writing file")


Writing newly formatted file...
Done writing file


In [9]:
with open(datafile, 'rb') as file:
    lines = file.readlines()
    for line in lines[:8]:
        print(line)

b"Can we make this quick?  Roxanne Korrine and Andrew Barrett are having an incredibly horrendous public break- up on the quad.  Again.\tWell I thought we'd start with pronunciation if that's okay with you.\r\r\n"
b"Well I thought we'd start with pronunciation if that's okay with you.\tNot the hacking and gagging and spitting part.  Please.\r\r\n"
b"Not the hacking and gagging and spitting part.  Please.\tOkay... then how 'bout we try out some French cuisine.  Saturday?  Night?\r\r\n"
b"You're asking me out.  That's so cute. What's your name again?\tForget it.\r\r\n"
b"No no it's my fault -- we didn't have a proper introduction ---\tCameron.\r\r\n"
b"Cameron.\tThe thing is Cameron -- I'm at the mercy of a particularly hideous breed of loser.  My sister.  I can't date until she does.\r\r\n"
b"The thing is Cameron -- I'm at the mercy of a particularly hideous breed of loser.  My sister.  I can't date until she does.\tSeems like she could get a date easy enough...\r\r\n"
b'Why?\tUnsolved 

In [10]:
PAD_token = 0 # used for padding short sentences
SOS_token = 1 # start of sentence token <START
EOS_token = 2 # end of sentence token <END>

class Vocabulary:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {PAD_token: "PAD", SOS_token: "SOS", EOS_token: "EOS"}
        self.num_words = 3

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.num_words
            self.word2count[word] = 1
            self.index2word[self.num_words] = word
            self.num_words += 1
        else:
            self.word2count[word] += 1

    def trim(self, min_count):
        keep_words = []
        for k, v in self.word2count.items():
            if v >= min_count:
                keep_words.append(k)

        print("keep words {} / {} = {:.4f}".format(len(keep_words), len(self.word2index), len(keep_words)/len(self.word2index)))
        self.word2index = {}
        self.word2count = {}
        self.index2word = {PAD_token: "PAD", SOS_token: "SOS", EOS_token: "EOS"}
        self.num_words = 3
        for word in keep_words:
            self.addWord(word)

In [11]:
def unicodeToAscii(s):
    return ''.join(c for c in unicodedata.normalize('NFD', s) if unicodedata.category(c) != 'Mn')

In [12]:
unicodeToAscii("Montréal,Françoise....")

'Montreal,Francoise....'

In [13]:
def normalizeString(s):
    s = unicodeToAscii(s.lower().strip())
    s = re.sub(r"([.!?])",r" ", s)
    s = re.sub(r"[^a-zA-Z.!?]+",r" ",s)
    s = re.sub(r"\s", r" ", s).strip()
    return s

In [14]:
normalizeString("aa123aa!s's  dd?")

'aa aa s s dd'

In [15]:
print("Reading and processing file....Please wait")
lines = open(datafile, 'r', encoding='utf-8').read().strip().split('\n')
pairs = [[normalizeString(s) for s in pair.split('\t')] for pair in lines ]
print("Done reading")
voc = Vocabulary("cornell movie-dialogs corpus")

Reading and processing file....Please wait
Done reading


In [16]:
MAX_LENGTH = 10
# return true if the pairs are under the threshhold
def filterPair(p):
    return len(p[0].split()) < MAX_LENGTH and len(p[1].split()) < MAX_LENGTH
def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]

In [17]:
pairs = [pair for pair in pairs if len(pair)>1]
print("There are {} pairs/conversations in the dataset)".format(len(pairs)))
pairs = filterPairs(pairs)
print("After filtering, there are {} pairs/conversations".format(len(pairs)))

There are 221275 pairs/conversations in the dataset)
After filtering, there are 84210 pairs/conversations


In [18]:
for pair in pairs:
    voc.addSentence(pair[0])
    voc.addSentence(pair[1])
print("Counted words:", voc.num_words)
for pair in pairs[:10]:
    print(pair)

Counted words: 21492
['gosh if only we could find kat a boyfriend', 'let me see what i can do']
['c esc ma tete this is my head', 'right see you re ready for the quiz']
['that s because it s such a nice one', 'forget french']
['there', 'where']
['you have my word as a gentleman', 'you re sweet']
['hi', 'looks like things worked out tonight huh']
['you know chastity', 'i believe we share an art instructor']
['have fun tonight', 'tons']
['well no', 'then that s all you had to say']
['then that s all you had to say', 'but']


In [19]:
MIN_COUNT = 3
def trimRareWords(voc,pairs, MIN_COUNT):
    voc.trim(MIN_COUNT)
    keep_pairs = []
    for pair in pairs:
        input_sentence = pair[0]
        output_sentence = pair[1]
        keep_input = True
        keep_output = True
        for word in input_sentence.split(' '):
            if word not in voc.word2index: 
                keep_input =False
                break
        for word in output_sentence.split(' '):
            if word not in voc.word2index: 
                keep_output =False
                break
        if keep_input and keep_output:
            keep_pairs.append(pair)
    print("Trimmed from {} pairs to {}, {:.4f} of total".format(len(pairs), len(keep_pairs),  len(keep_pairs)/len(pairs)))
    return keep_pairs

pairs = trimRareWords(voc,pairs,MIN_COUNT)

keep words 9978 / 21489 = 0.4643
Trimmed from 84210 pairs to 71445, 0.8484 of total


In [20]:
def indexesFromSentence(voc, sentence):
    return [voc.word2index[word] for word in sentence.split(' ')] + [EOS_token]

In [21]:
indexesFromSentence(voc, pairs[1][0])

[31, 32, 33, 34, 32, 35, 10, 36, 37, 2]

In [22]:
inp = []
out = []
i = 0
for pair in pairs[:10]:
    inp.append(pair[0])
    out.append(pair[1])
print(inp)
print(len(inp))
indexes = [indexesFromSentence(voc, sentence) for sentence in inp]
indexes

['gosh if only we could find kat a boyfriend', 'that s because it s such a nice one', 'there', 'you have my word as a gentleman', 'hi', 'have fun tonight', 'well no', 'then that s all you had to say', 'but', 'do you listen to this crap']
10


[[3, 4, 5, 6, 7, 8, 9, 10, 11, 2],
 [31, 32, 33, 34, 32, 35, 10, 36, 37, 2],
 [40, 2],
 [26, 42, 23, 43, 44, 10, 45, 2],
 [47, 2],
 [42, 60, 53, 2],
 [62, 63, 2],
 [64, 31, 32, 65, 26, 66, 67, 68, 2],
 [69, 2],
 [18, 26, 73, 67, 21, 74, 2]]

In [23]:
def zeroPadding(l, fillvalue = 0):
    return list(itertools.zip_longest(*l, fillvalue=fillvalue))

In [24]:
leng = [len(ind) for ind in indexes]
max(leng)

10

In [25]:
test_result = zeroPadding(indexes)
print(len(test_result))
test_result

10


[(3, 31, 40, 26, 47, 42, 62, 64, 69, 18),
 (4, 32, 2, 42, 2, 60, 63, 31, 2, 26),
 (5, 33, 0, 23, 0, 53, 2, 32, 0, 73),
 (6, 34, 0, 43, 0, 2, 0, 65, 0, 67),
 (7, 32, 0, 44, 0, 0, 0, 26, 0, 21),
 (8, 35, 0, 10, 0, 0, 0, 66, 0, 74),
 (9, 10, 0, 45, 0, 0, 0, 67, 0, 2),
 (10, 36, 0, 2, 0, 0, 0, 68, 0, 0),
 (11, 37, 0, 0, 0, 0, 0, 2, 0, 0),
 (2, 2, 0, 0, 0, 0, 0, 0, 0, 0)]

In [26]:
def binaryMatrix(l, values=0):
    m = []
    for i, seq in enumerate(l):
        m.append([])
        for token in seq:
            if token == PAD_token: 
                m[i].append(0)
            else: 
                m[i].append(1)
    return m

In [27]:
binary_result = binaryMatrix(test_result)
binary_result

[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 0, 1, 0, 1, 1, 1, 0, 1],
 [1, 1, 0, 1, 0, 1, 0, 1, 0, 1],
 [1, 1, 0, 1, 0, 0, 0, 1, 0, 1],
 [1, 1, 0, 1, 0, 0, 0, 1, 0, 1],
 [1, 1, 0, 1, 0, 0, 0, 1, 0, 1],
 [1, 1, 0, 1, 0, 0, 0, 1, 0, 0],
 [1, 1, 0, 0, 0, 0, 0, 1, 0, 0],
 [1, 1, 0, 0, 0, 0, 0, 0, 0, 0]]

In [28]:
# return padded input sequence tensor and as well as a tensor of lengths for each of the sequences in the batch
def inputVar(l, voc):
    indexes_batch = [indexesFromSentence(voc, sentence) for sentence in l]
    lengths = torch.tensor([len(indexes) for indexes in indexes_batch])
    padList = zeroPadding(indexes_batch)
    padVar = torch.LongTensor(padList)
    return padVar, lengths

In [29]:
# returns the padded target sequence tensor, paddinbg mask and max target length
def outputVar(l, voc):
    indexes_batch = [indexesFromSentence(voc, sentence) for sentence in l]
    max_target_len = max([len(indexes) for indexes in indexes_batch])
    padList = zeroPadding(indexes_batch)
    mask = binaryMatrix(padList)
    mask = torch.ByteTensor(mask)
    padVar = torch.LongTensor(padList)
    return padVar, mask, max_target_len

In [30]:
# return all items for a given batch of pairs
def batch2TrainData(voc, pair_batch):
    pair_batch.sort(key=lambda x: len(x[0].split(" ")), reverse=True)
    input_batch, output_batch = [], []
    for pair in pair_batch:
        input_batch.append(pair[0])
        output_batch.append(pair[1])
    inp, lengths = inputVar(input_batch,voc)
    output, mask, max_target_len = outputVar(output_batch, voc)
    return inp, lengths, output, mask, max_target_len

In [31]:
small = 5
batches = batch2TrainData(voc, [random.choice(pairs) for _ in range(small)])
inputvar, lengths, targetvar, mask, maxtar = batches
print("input variable:")
print(inputvar)
print("lengths variable:")
print(lengths)
print("target variable:")
print(targetvar)
print("max variable:")
print(maxtar)
print("mask variable:")
print(mask)

input variable:
tensor([[ 187,  111,   68,   62,  165],
        [ 449,  696,  317,   91,   62],
        [ 269,   26,  115, 1404,   41],
        [  18,  157,   21,   32,   18],
        [  26,   67,   91,   88,    6],
        [  42,   42,  112,   86,  335],
        [1028,  457,  685,   75,    2],
        [ 221, 2462,    2,    2,    0],
        [  34,    2,    0,    0,    0],
        [   2,    0,    0,    0,    0]])
lengths variable:
tensor([10,  9,  8,  8,  7])
target variable:
tensor([[  16,   16,   16,  187,  333],
        [ 114,  114,   95,   18,   26],
        [ 115,  115,   34,   26,   27],
        [  55,    2,   22,   55,   30],
        [ 372,    0,    2,    2,  201],
        [ 113,    0,    0,    0,  599],
        [5788,    0,    0,    0,    2],
        [   2,    0,    0,    0,    0]])
max variable:
8
mask variable:
tensor([[1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1],
        [1, 0, 1, 1, 1],
        [1, 0, 0, 0, 1],
        [1, 0, 0,

In [32]:
class EncoderRNN(nn.Module):
    def __init__(self, hidden_size, embedding, n_layers=1, dropout=0):
        self.n_layers = n_layers
        self.hidden_size = hidden_size
        self.embedding = embedding
        # initlize the GRU, input size and hidden size are set to hidden_size
        self.gru = nn.GRU(hidden_size, hidden_size, n_layers, dropout=(0 if n_layers == 1 else dropout), bidirectional=True)

    def forward(self, input_seq, input_lengths, hidden=None):
        # input_seq: batch of input sentences
        # input_lenghts: kist of sentence lengths
        # hidden state, of shape (n_layers * num_directions, batch_size hidden_size)
        # convert word indexes to embedding
        embedded = self.embedding(input_seq)
        # pack padded batch of sequences for RNN module
        packed = torch.nn.utils.rnn.pack_padded_sequence(embedded, input_lengths)
        # forward pass
        outputs, hidden =self.gru(packed,hidden)
        # unpack padding 
        outputs, _ = torch.nn.utils.rnn.pad_packed_sequence(outputs)
        # sum bidirectional GRU outputs
        outputs = outputs[:, :, :self.hidden_size] + outputs[:, :, self.hidden_size:]
        # outputs: output feature h_t from tha last layer
        # hidfden: hidden state for the last timestamp
        return outputs, hidden

In [34]:
class Attn(torch.nn.Module):
    def __init__(self, method, hidden_size):
        super(Attn, self).__init__()
        self.method = method
        self.hidden_siez = hidden_size

    def dot_score(self, hidden, encoder_output):
        # element wise multiply the current target decoder state with the encoder output and sum them
        return torch.sum(hidden * encoder_output, dim=2)

    def forward(self, hidden, encoder_outputs):
        # hidden of shape: (1, batch size, hidden size)
        # encoder outputs of shape: (max length, barch size, hidden size)

        # calculate the attention weights
        attn_energies = self.dot_score(hidden, encoder_outputs)
        # transpose max length and batch size dimensions
        attn_energies = attn_energies.t()
        return F.softmax(attn_energies, dim=1).unsqueeze(1)

In [41]:
class LoungAttnDecoderRNN(nn.Module):
    def __init__(self, attn_model, embedding, hidden_size, output_size, n_layers, dropout=0.1):
        super(LoungAttnDecoderRNN, self).__init__()
        self.attn_model = attn_model
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.n_layers = n_layers
        self.dropout = dropout

        self.embedding = embedding
        self.embedding_dropout = nn.Dropout(dropout)
        self.gru = nn.GRU(hidden_size, hidden_size, n_layers, dropout=(0 if n_layers == 1 else dropout))
        self.concat = nn.Linear(hidden_size * 2, output_size)

        self.attn = Attn(attn_model, hidden_size)

    def forward(self, input_step, last_hidden, encoder_outputs):
        # input step: one last step of input sequence batch
        # last hidden: final hidden layer of GRU
        # encoder outputs: encoder models output
        #get embedding of current input word
        embedded = self.embedding(input_step)
        embedded = self.embedding_dropout(embedded)
        # forward through undirictional GRU
        rnn_output, hidden = self.gru(embedded, last_hidden)
        # calculate attention weights from current GRU output
        attn_weights = self.attn(rnn_output, encoder_outputs)
        # multiply attention weights to encoder to get new weighted sum context vector
        context = attn_weights.bmm(encoder_outputs.transpose(0,1))
        # concatenate weighted context vector and GRU output
        rnn_output = rnn_output.squeeze(1)
        context = context.squeeze(1)
        concat_input = torch.cat((rnn_output, context),1)
        concat_output = torch.tanh(self.concat(concat_input))

        output = self.out(concat_output)
        output = F.softmax(output, dim=1)
        return output, hidden

In [42]:
def maskNLLLose(decoder_out, target, mask):
    nTotal = mask.sum
    target = target.view(-1,1)
    gathered_tensor = torch.gather(decoder_out,1, target)
    crossEntropy = - torch.log(gathered_tensor)
    loss = crossEntropy.masked_select(mask)
    loss = loss.mean()
    loss = loss.to(device)
    return loss, nTotal.item()

In [43]:
small_batch_size = 5
batches = batch2TrainData(voc, [random.choice(pairs) for _ in range(small_batch_size)])
input_variable, lenghts, target_variable, mask, max_target_len = batches

print("input variable shape:", input_variable.shape)
print("lengths shape:", lenghts.shape)
print("target variable shape:", target_variable.shape)
print("mask shape:", mask.shape)
print("max_target_len shape:", max_target_len)

hidden_size = 500
encoder_n_layers = 2
decoder_n_layers = 2
dropout = 0.1
attn_model = 'dot'
embedding = nn.Embedding(voc.num_words, hidden_size)

# define encoder and decoder


input variable shape: torch.Size([9, 5])
lengths shape: torch.Size([5])
target variable shape: torch.Size([10, 5])
mask shape: torch.Size([10, 5])
max_target_len shape: 10
